In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [4]:
import os
print(os.getcwd())

/home/angelo/Documents/Uni/Courses/Digital Transformation & Supply Chain/supply_chain_assignment


## Load the data


In [75]:
df_customer_Churn = pd.read_csv("Data/Daily_Churned_customers_JET.csv")

df_input_accounts = pd.read_csv("Data/input_accounts_21_10_2022.csv")

df_tweets = pd.read_csv("Data/tweets-2022-10-21_justeattakeaway.csv")

#### Churn Change

In [76]:
df_customer_Churn["d_churn"] = df_customer_Churn["Churned customers"].shift(1) - df_customer_Churn["Churned customers"]

In [77]:
df_customer_Churn

,Day,Churned customers,d_churn
0,1/1/2021,5757881,NaN
1,2/1/2021,5760714,-2833.0
2,3/1/2021,5765953,-5239.0
3,4/1/2021,5761176,4777.0
4,5/1/2021,5755861,5315.0
...,...,...,...
664,27/10/2022,8831343,-7003.0
665,28/10/2022,8828483,2860.0
666,29/10/2022,8850094,-21611.0
667,30/10/2022,8853747,-3653.0


### Part 2. Data analysis of the platform’s Twitter data

For the next part of the assignment you would be asked to put yourself in the shoes of the data analyst working at the platform JustEatTakeaway.com. You are asked by the platform management to explore whether the social media data could be useful for the platform to learn how the social media users (and potential customers) perceive the announcements about different innovations. Specifically, the platform is active in several accounts on Twitter regularly posting news and messages and replying to customers. You are asked to find out what could the platform management learn about users’ perception of its innovations from the user engagement with the platform’s tweets.

For that, you are provided with the data on daily customer churn from JustEatTakeaway.com and tweets from the Twitter accounts of JustEatTakeaway and their competitors, Deliveroo and UberEats. The customer churn data covers the period of 01.2021 - 10.2022 and tweets cover the period of 11.2010-10.2022.

The data analysis should be performed using python, for example, the libraries for data manipulation (pandas), descriptive analysis (pandas), graphical analysis (seaborn), regressions (statsmodels) and sentiment analysis (nltk/vader). However, you may choose any other suitable libraries.

You may choose any relevant methods for your descriptive analysis and / or regression analysis.

You are welcome to do feature engineering and if you create more than 5 additional features (independent variables) you may get a bonus of 0.5 points.*Creating a set of dummies for one variable counts as one feature, for example, the set of year dummies counts as one feature engineered.



### Please, address the following questions:

 

Describe the platform’s strategy on Twitter:
2.1 What are the main topics the company tweets about? What are the most used words / hashtags / bigrams (trigrams)? How often and in which moments of time does the platform post tweets most often?* (1 slide)

*Before you analyze the texts of tweets, please, preprocess the texts of tweets. You can find the list of simple preprocessing steps below, but you may also do any more sophisticated preprocessing (upon your choice).

2.2 Please, compute the sentiment scores for each tweet using nltk (vader) library or any other library of your choice. You may also engineer any additional variables based on computed sentiment scores. How could you describe the platform’s tweets based on their sentiment scores? (1 slide)

 

What features of the platform’s tweets are related to higher / lower user engagement with the tweets as measured by the number of  likes, retweets, or replies per tweet? To answer this question, please, run the regression analysis. As dependent variables, you may use the number of likes, retweets and / or replies to each tweet of the platform. As independent variables, you may use any relevant features from the Twitter data set, any additional features you created, and the sentiment scores from point 2.2 (or any other features you engineered based on sentiment scores).


3.1 Is the use of positive / negative emotional words in the company’s tweets (original posts or replies) related to higher / lower user engagement as measured by the number of  likes, retweets? (1 slide)

3.2 Do your findings hold for any time period or can you observe any outstanding time or seasonal patterns? For your analysis you may check any periods of interest, for example, covid-19 lockdowns, summer months, holidays, weekly patterns, time of the day patterns. (1 slide)

3.3 Please, check whether user engagement with tweets of JustEatTakeaway.com is associated (or correlated) with customer churn? Do you observe any temporal patterns in this association, i.e. does this association differ depending what period you consider? How can you interpret the observed associations? (1 slide)



# Part 2. Data analysis of the platform’s Twitter data

## data Preprocessing

##### 1. Remove all non-character signs and stop words (Stop words are a set of commonly used words in a language that do not have any meaning. Examples of stop words in English are “a”, “the”, “is”, “are” and etc.):

In [78]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from termcolor import colored
import math

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [79]:
def show_vocabulary(vectorizer, word_size=15, words_per_line=10):
    words = vectorizer.get_feature_names()

    print(f'Vocabulary size: {len(words)} words')

    word_format = f'<{word_size}'
    for l in np.array_split(words, math.ceil(len(words) / words_per_line)):
        print(''.join([f'{x:{word_format}}' for x in l]))


def show_bow(vectorizer, bow, word_size=15, words_per_line=8):
    words = vectorizer.get_feature_names()

    word_format = f'<{word_size}'
    for l in np.array_split(list(zip(words, bow)), math.ceil(len(words) / words_per_line)):
        print(' | '.join([colored(f'{w:{word_format}}:{n:>2}', 'grey') if int(n) == 0 else colored(f'{w:{word_format}}:{n:>2}', on_color='on_yellow', attrs=['bold']) for w, n in l ]))

def show_bow_float(vectorizer, bow, word_size=15, words_per_line=6):
    words = vectorizer.get_feature_names()

    word_format = f'<{word_size}'
    for l in np.array_split(list(zip(words, bow)), math.ceil(len(words) / words_per_line)):
        print(' | '.join([colored(f'{w:word_format}:{float(n):>0.2f}', 'grey') if float(n) == 0 else colored(f'{w:word_format}:{float(n):>0.2f}', on_color='on_yellow', attrs=['bold']) for w, n in l ]))


In [118]:
additional  = ['rt','rts','retweet'] #we'll store additional stopwords here

swords = set().union(stopwords.words(['dutch', "english"]),additional)

#replace all signs and stopwords in tweets and tokenize tweets (tokenization means splitting up a larger body of text into pieces such as words, keywords, phrases, symbols and other elements called tokens; in our case we can split the tweets into words)
df_tweets.drop_duplicates(subset='text',inplace=True)




0                           @FoodNetwork flats for the win
1        @PopTartsUS that's too many flavors at once bro 🤔
2                                    @AuntieAnnes retweet.
3        @_MarieGrey iS uR rEfRiGeRaToR rUNniNg BeCAuSe...
4        @Drizly extra points for putting more water in...
                               ...                        
18591    @panikosgeor @Deliveroo @UberEats Hi Panikos, ...
18592            @Gerdav74 Eet smakelijk alvast! 🍣 ^Isabel
18593    @JanKarens @tonnyw @Kwalitaria Toch wil ik je ...
18594    @MartinMunSigra Hee tas! Waar ga je met die be...
18595    @bezemsteeI Hmmm, okay.. Ik kan helaas hier ni...
Name: text, Length: 18567, dtype: object

In [119]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

df_tweets['text_lemmatized']
df_tweets['text_lemmatized'] = df_tweets.text.apply(lemmatize_text)

NameError: name 'nltk' is not defined

In [117]:



df_tweets['processed_text'] = df_tweets['text'].str.lower()\
            .str.replace('(@[a-z0-9]+)\w+',' ')\
            .str.replace('(http\S+)', ' ')\
            .str.replace('([^0-9a-z \t])',' ')\
            .str.replace(' +',' ')\
            .apply(lambda x: [i for i in x.split() if not i in swords])

/tmp/ipykernel_23819/390628904.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('(@[a-z0-9]+)\w+',' ')\
/tmp/ipykernel_23819/390628904.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('(http\S+)', ' ')\
/tmp/ipykernel_23819/390628904.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('([^0-9a-z \t])',' ')\
/tmp/ipykernel_23819/390628904.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace(' +',' ')\


In [93]:

df_tweets['processed_text'] 

0                                             [flats, win]
1                                     [many, flavors, bro]
2                                                       []
3        [mariegrey, ur, refrigerator, running, lol, jk...
4            [extra, points, putting, water, bowl, cheers]
                               ...                        
18591    [hi, panikos, restaurants, indicate, use, deli...
18592                     [eet, smakelijk, alvast, isabel]
18593    [vragen, even, dm, sturen, hiervoor, open, sta...
18594            [hee, tas, waar, ga, bezorger, toe, iris]
18595    [hmmm, okay, helaas, terugvinden, voucher, zom...
Name: processed_text, Length: 18567, dtype: object

##### Propper preprocessing using a count vectorizer 

##### Filtering by Document Frequency

In [115]:
count = CountVectorizer(
    stop_words=swords,
    token_pattern=r'[a-z]+\w*',
    max_features=50000,
    min_df=5,
    max_df=0.8
)
tf = count.fit_transform(df_tweets.text)
count.vocabulary_
vocab_df = count.get_feature_names()
print(f'Size of vocabulary: {len(vocab_df)}')

Size of vocabulary: 3513


In [116]:
tf.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
count = CountVectorizer()
# count.fit(df_tweets.text)
# show_vocabulary(count)
tf = count.fit_transform(df_tweets['text'])
count.vocabulary_
count.get_feature_names()
tf.toarray()

#### Stemming/Lemmatizing

In [ ]:

nltk.download('wordnet')

In [ ]:


wn = WordNetLemmatizer()
ps = PorterStemmer()


In [71]:

count = CountVectorizer(
    stop_words='english',
    ngram_range=(1,3),    # will create a vocabulary with 1-gram and 2-grams
    min_df=2,
    max_df=0.8,
    max_features=1500
)

count.fit(df_tweets.text)

CountVectorizer(max_df=0.8, max_features=1500, min_df=2, ngram_range=(1, 3),
                stop_words='english')

In [72]:
show_vocabulary(count, word_size=25, words_per_line=6)

Vocabulary size: 1500 words
000                      10                       100                      19uzbmarzd               20                       9hqhr1pvi1               
aan                      aan het                  aan het wachten          able                     able drop                able drop dm             
able follow              able follow request      able resolve             able resolve effectively account                  account additional       
account additional detailsaccount alternatively    account alternatively contactaccount assist           account assist better    account details          
account details email    account dm               account email            account email address    account look             account order            
account order id         account screenshot       account screenshot showingaccount trip             account trip details     act                      
act responsibly          act responsibly respectfullyadditio

#### 2. Stemming - an algorithm to categorize similar words (or words with the same roots) into one. We can use the porter stemmer.

In [26]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

df_tweets['stemmed'] = df_tweets['processed_text'].apply(lambda x: [ps.stem(i) for i in x if i != ''])

df_tweets['stemmed'].head(n=10)

0                                          [flat, win]
1                                  [mani, flavor, bro]
2                                                   []
3    [mariegrey, ur, refriger, run, lol, jk, happi,...
4              [extra, point, put, water, bowl, cheer]
5                                  [gonna, say, thing]
6                       [ok, got, earnest, lol, outta]
7                   [liter, morn, meet, w, boss, week]
8                                   [watch, one, hand]
9                                       [icon, around]
Name: stemmed, dtype: object

#### sentiments 

In [27]:
import nltk.sentiment.vader as vd
from nltk import download
download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/angelo/nltk_data...


True

In [29]:
sia = vd.SentimentIntensityAnalyzer()
from nltk.tokenize import word_tokenize
df_tweets['sentiment_score'] = df_tweets['processed_text'].apply(lambda x: sum([ sia.polarity_scores(i)['compound'] for i in word_tokenize( ' '.join(x) )]) )
df_tweets[['processed_text','sentiment_score']].head(n=10)
df_tweets['sentiment_score'].apply(lambda x: round(x,)).value_counts()

 0    9762
 1    4657
 2    3745
-1     181
 3     171
 4      44
 5       3
 6       2
-2       2
Name: sentiment_score, dtype: int64

### 2.1 What are the main topics the company tweets about? What are the most used words / hashtags / bigrams (trigrams)? How often and in which moments of time does the platform post tweets most often?* (1 slide)

wordcloud, n grams, LDA, SDA, show vocabulary and also show vocabulary ba document via LDA

## 2.2 Please, compute the sentiment scores for each tweet using nltk (vader) library or any other library of your choice. You may also engineer any additional variables based on computed sentiment scores. How could you describe the platform’s tweets based on their sentiment scores? (1 slide)

here we should find the sentiment score of each document; classify it and then build a regression on this 